This notebook compares the ```depth (variance)``` and ```PSF``` of ```coadds (templates)``` and ```difference images``` from two butler collections.

The idea behind is we expect to see more variance in the good template than deep template. But, the PSF is better for the good than deep. 

1. At which point can we say using one is better over using the other?

incollection = LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20231207/w_2023_49/PREOPS-4648

```good seeing template :``` 
include only good seeing visits to create the template.

collection_good = f"u/jarugula/auxtel_goodseeing_template_subtract"

```deepcoadd template :```
includes all the visits to create the template. This is done by adding the following code in the pipeline yaml 

```
tasks:
    selectGoodSeeingVisits:
        class: lsst.pipe.tasks.selectImages.BestSeeingQuantileSelectVisitsTask
        config:
          connections.goodVisits: goodSeeingVisits
          qMin: 0
          qMax: 0.99
    
```
collection_deep = f"u/jarugula/auxtel_deep_template_subtract"



All the Plots and tables are in PREOPS-4952-depthcomparision.ipynb

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gc
import random 

from astropy.coordinates import SkyCoord
import astropy.units as u

import lsst.daf.butler as dafButler
from lsst.pex.config import Config
from lsst.pipe.base import Instrument
import lsst.afw.display as afwDisplay
import lsst.display.astrowidgets

afwDisplay.setDefaultBackend('astrowidgets')
afwDisplay.setDefaultBackend('matplotlib')
plt.style.use('tableau-colorblind10')
%matplotlib inline

### The two butler collections

In [2]:
repo = '/sdf/group/rubin/repo/embargo'
skymapName = 'latiss_v1'
instrumentName = 'LATISS'

collection_good = f"u/{os.environ['USER']}/auxtel_goodseeing_template_subtract"
collection_deep = f"u/{os.environ['USER']}/auxtel_deep_template_subtract"

butler_good = dafButler.Butler(repo, collections=collection_good, skymap=skymapName)
butler_deep = dafButler.Butler(repo, collections=collection_deep, skymap=skymapName)

registry_good = butler_good.registry
registry_deep = butler_deep.registry

### Selecting the dataids with varying number of visits going into making the good and deep templates

There are more visits in the deep than good templates.

Randomly select 10 visits from the vists going into making the coadds. We compare the variance and PSF of these visits

In [3]:
dataid_template_list = [{'band': 'r', 'skymap': 'latiss_v1', 'tract': 8188, 'patch': 65},
                        {'band': 'g', 'skymap': 'latiss_v1', 'tract': 10644, 'patch': 183},
                        {'band': 'i', 'skymap': 'latiss_v1', 'tract': 3863, 'patch': 262},
                        {'band': 'y', 'skymap': 'latiss_v1', 'tract': 383, 'patch': 359},
                        {'band': 'y', 'skymap': 'latiss_v1', 'tract': 455, 'patch': 14},
                        {'band': 'i', 'skymap': 'latiss_v1', 'tract': 5614, 'patch': 211},
                        {'band': 'i', 'skymap': 'latiss_v1', 'tract': 382, 'patch': 342},
                        {'band': 'r', 'skymap': 'latiss_v1', 'tract': 455, 'patch': 33},
                        {'band': 'g', 'skymap': 'latiss_v1', 'tract': 3864, 'patch': 254},
                        {'band': 'i', 'skymap': 'latiss_v1', 'tract': 5634, 'patch': 74},
                        {'band': 'r', 'skymap': 'latiss_v1', 'tract': 5634, 'patch': 93},
                        {'band': 'g', 'skymap': 'latiss_v1', 'tract': 3864, 'patch': 256},
                        {'band': 'g', 'skymap': 'latiss_v1', 'tract': 5634, 'patch': 92},
                        {'band': 'r', 'skymap': 'latiss_v1', 'tract': 5634, 'patch': 92}]

good_template_visits = [1, 2, 5, 4, 5, 8, 16, 22, 25, 34, 47, 63, 102, 119]
deep_template_visits = [1, 2, 5, 12, 17, 21, 41, 54, 79, 97, 137, 195, 265, 350]



In [4]:
print('Selecting random 10 visits from the visits in goodseeing template')
visit_list = []
for i, dataId in enumerate(dataid_template_list):
    # print(dataId)
    # print('Number of visits in good template: ', good_template_visits[i])
    # print('Number of visits in deep template: ', deep_template_visits[i])
    exposure_good = butler_good.get('goodSeeingCoadd', dataId=dataId)
    visits = exposure_good.getInfo().getCoaddInputs().visits["id"].tolist()
    if len(set(visits)) >= 10:
        random_visits = random.sample(visits, 10)
    else:
        random_visits = visits
    # print(random_visits)
    visit_list.append(random_visits)
    # print('----------------')

Selecting random 10 visits from the visits in goodseeing template


e.g.

![alt text](./PREOPS-4952-figures/select_visits_good_deep.png "Title")

The DIA sources from ```goodSeeingDiff_diaSrcTable``` from the selected visits

![alt text](./PREOPS-4952-figures/dia_src_example.png "Title")

In [ ]:
visit_list_all = [x for xs in visit_list for x in xs]
num_visits_good = []
num_visits_deep = []

for i, dataId_list in enumerate(dataId_list_all_templates):
    for dataId in dataId_list:
        num_visits_good.append(good_template_visits[i])
        num_visits_deep.append(deep_template_visits[i])  

## Computing the Variance of good and deep templates and the difference images

The psf determinant radius is computed

[https://community.lsst.org/t/image-difference-task-using-decam-images-gen3/5947/27](https://community.lsst.org/t/image-difference-task-using-decam-images-gen3/5947/27)

[https://pipelines.lsst.io/py-api/lsst.afw.geom.Quadrupole.html#lsst.afw.geom.Quadrupole](https://pipelines.lsst.io/py-api/lsst.afw.geom.Quadrupole.html#lsst.afw.geom.Quadrupole)


In [ ]:
# Plotting calexp, good template and difference image variance

sigma2fwhm = 2.*np.sqrt(2.*np.log(2.))

median_calexp_good = []
median_template_good = []
median_diff_good = []

psf_radius_calexp_good = []
psf_radius_template_good = []
psf_radius_diff_good = []

for i, dataId_list in enumerate(dataId_list_all_templates):
    print('Number of visits in good template: ', good_template_visits[i])

    for dataId in dataId_list:
        dataId = dataId.mapping
        dataId_string = {str(key): dataId[key] for key in dataId.keys()}
        
        calexp_image_good = butler_good.get('calexp', **dataId_string).variance
        diff_image_good = butler_good.get('goodSeeingDiff_differenceExp', **dataId_string).variance
        template_image_good = butler_good.get('goodSeeingDiff_templateExp', **dataId_string).variance

        calexp_image_psf = butler_good.get('calexp', **dataId_string).psf
        calexp_psf_radius = calexp_image_psf.computeShape(calexp_image_psf.getAveragePosition()).getDeterminantRadius()*sigma2fwhm
        
        diff_image_psf = butler_good.get('goodSeeingDiff_differenceExp', **dataId_string).psf
        diff_psf_radius = diff_image_psf.computeShape(diff_image_psf.getAveragePosition()).getDeterminantRadius()*sigma2fwhm
        
        template_image_psf = butler_good.get('goodSeeingDiff_templateExp', **dataId_string).psf
        template_psf_radius = template_image_psf.computeShape(template_image_psf.getAveragePosition()).getDeterminantRadius()*sigma2fwhm

        
        psf_radius_calexp_good.append(calexp_psf_radius)
        psf_radius_template_good.append(template_psf_radius)
        psf_radius_diff_good.append(diff_psf_radius)
    

        image_arr = template_image_good.array
        median_template = round(np.nanmedian(image_arr), 3)
        median_template_good.append(median_template)
    
        image_arr = calexp_image_good.array
        median_calexp = round(np.nanmedian(image_arr), 3)
        median_calexp_good.append(median_calexp)

        image_arr = diff_image_good.array
        median_diff = round(np.nanmedian(image_arr), 3)
        median_diff_good.append(median_diff)

        del(dataId_string)

In [ ]:
# Plotting calexp, deep template and difference image variance

median_calexp_deep = []
median_template_deep = []
median_diff_deep = []

psf_radius_calexp_deep = []
psf_radius_template_deep = []
psf_radius_diff_deep = []

for i, dataId_list in enumerate(dataId_list_all_templates):
    print('Number of visits in deep template: ', deep_template_visits[i])

    for dataId in dataId_list:
        dataId = dataId.mapping
        dataId_string = {str(key): dataId[key] for key in dataId.keys()}
        calexp_image_deep = butler_deep.get('calexp', **dataId_string).variance
        diff_image_deep = butler_deep.get('goodSeeingDiff_differenceExp', **dataId_string).variance
        template_image_deep = butler_deep.get('goodSeeingDiff_templateExp', **dataId_string).variance

        calexp_image_psf = butler_deep.get('calexp', **dataId_string).psf
        calexp_psf_radius = calexp_image_psf.computeShape(calexp_image_psf.getAveragePosition()).getDeterminantRadius()*sigma2fwhm
        
        diff_image_psf = butler_deep.get('goodSeeingDiff_differenceExp', **dataId_string).psf
        diff_psf_radius = diff_image_psf.computeShape(diff_image_psf.getAveragePosition()).getDeterminantRadius()*sigma2fwhm
        
        template_image_psf = butler_deep.get('goodSeeingDiff_templateExp', **dataId_string).psf
        template_psf_radius = template_image_psf.computeShape(template_image_psf.getAveragePosition()).getDeterminantRadius()*sigma2fwhm

        psf_radius_calexp_deep.append(calexp_psf_radius)
        psf_radius_template_deep.append(template_psf_radius)
        psf_radius_diff_deep.append(diff_psf_radius)

        image_arr = template_image_deep.array
        median_template = round(np.nanmedian(image_arr), 3)
        median_template_deep.append(median_template)
    
        image_arr = calexp_image_deep.array
        median_calexp = round(np.nanmedian(image_arr), 3)
        median_calexp_deep.append(median_calexp)
    
        image_arr = diff_image_deep.array
        median_diff = round(np.nanmedian(image_arr), 3)
        median_diff_deep.append(median_diff)
    
        del(dataId_string)

In [ ]:
median_dict = {'num of visits in good coadd': num_visits_good,
               'num of visits in deep coadd': num_visits_deep,
               'visit' : visit_list_all,
               'median calexp variance' : median_calexp_good,
               'median template variance good' : median_template_good,
               'median template variance deep' : median_template_deep,
               'median diff variance good' : median_diff_good,
               'median diff variance deep' : median_diff_deep,
               'calexp psf' : psf_radius_calexp_deep,
               'template psf good': psf_radius_template_good,
               'template psf deep': psf_radius_template_deep,
               'diff psf good': psf_radius_diff_good,
               'diff psf deep': psf_radius_diff_deep,
              }
median_df = pd.DataFrame.from_dict(median_dict)
median_df.to_csv('median_variance_psf_10visits_per_coadd.csv')

In [5]:
bins=np.linspace(5, 25, 20)
plt.hist(psf_radius_calexp_deep, bins=bins, alpha=0.8, edgecolor='orange', label='Deep', facecolor="None")
plt.hist(psf_radius_calexp_good, bins=bins, alpha=0.8, edgecolor='blue', label='Good', facecolor="None")
plt.xlabel('PSF radius')
plt.title('PSF historam of Calexp images')
plt.legend()

bins=np.linspace(0, 10, 20)
plt.hist(median_template_good, bins=bins, alpha=0.8, edgecolor='orange', label='Good', facecolor="None")
plt.hist(median_template_deep, bins=bins, alpha=0.8, edgecolor='blue', label='Deep', facecolor="None")
plt.xlabel('Variance')
plt.title('Median Variance of Templates')
plt.legend()

plt.hist(median_diff_good, alpha=0.8, edgecolor='orange', label='Good', facecolor="None")
plt.hist(median_diff_deep, alpha=0.8, edgecolor='blue', label='Deep', facecolor="None")
plt.xlabel('Variance')
plt.title('Median Variance of Difference Images')
plt.legend()

NameError: name 'psf_radius_calexp_deep' is not defined

![alt text](./PREOPS-4952-figures/median_variance_calexp.png "Title")

![alt text](./PREOPS-4952-figures/median_variance_templates.png "Title")

![alt text](./PREOPS-4952-figures/median_variance_diff.png "Title")


The variance of the good templates is slightly larger than the deep templates. 

This is expected as there are fewer visits going into making the good template than the deep template

### PSF

For the different methods of convolution refer to the notes

[https://dmtn-256.lsst.io](https://dmtn-256.lsst.io)

The default convolution algorithm adjusts the PSF of the templates (if the PSF of template is larger) to match the PSF of the calexp.

The PSF of the deep templates is larger (worse) than the good templates as expected due to more visits going into making it. 

However, the PSF of the difference images is similar no matter which template we use. This is probably because of the convolution algorithm used to match the PSFs to the PSF of calexp.

In [ ]:
bins=np.linspace(5, 25, 20)
plt.hist(psf_radius_calexp_deep, bins=bins, alpha=0.8, edgecolor='orange', label='Deep', facecolor="None")
plt.hist(psf_radius_calexp_good, bins=bins, alpha=0.8, edgecolor='blue', label='Good', facecolor="None")
plt.xlabel('PSF radius')
plt.title('PSF historam of Calexp images')
plt.legend()

plt.hist(psf_radius_template_deep, bins=bins, alpha=0.8, edgecolor='orange', label='Deep', facecolor="None")
plt.hist(psf_radius_template_good,bins=bins,  alpha=0.8, edgecolor='blue', label='Good', facecolor="None")
plt.xlabel('PSF radius')
plt.title('PSF historam of Template images')
plt.legend()

plt.hist(psf_radius_diff_deep, bins=bins, alpha=0.8, edgecolor='orange', label='Deep', facecolor="None")
plt.hist(psf_radius_diff_good, bins=bins, alpha=0.8, edgecolor='blue', label='Good', facecolor="None")
plt.xlabel('PSF radius')
plt.title('PSF historam of Difference images: Default convolution to match to calexp PSF')
plt.legend()

![alt text](./PREOPS-4952-figures/psf_calexp.png "Title")

![alt text](./PREOPS-4952-figures/psf_templates.png "Title")

![alt text](./PREOPS-4952-figures/psf_diff.png "Title")

In [ ]:
plt.scatter(median_df['calexp psf'], median_df['template psf good'])
plt.plot([9,20], [9, 20])
plt.xlabel('Calexp PSF')
plt.ylabel('Good Template PSF')

plt.scatter(median_df['calexp psf'], median_df['template psf deep'])
plt.plot([9,20], [9, 20])
plt.xlabel('Calexp PSF')
plt.ylabel('Deep Template PSF')

![alt text](./PREOPS-4952-figures/psf_good_calexp.png "Title")

![alt text](./PREOPS-4952-figures/psf_deep_calexp.png "Title")


###Selecting visits where good template psf < visit psf < deep template psf 

fraction of visits with good template psf < visit psf < deep template psf :  0.524

fraction of visits with visit psf < good and deep template:  0.409

fraction of visits with deep template psf < visit psf < good template psf:  0.0

In [ ]:
df_interesting_visits =  median_df[(median_df['calexp psf'] > median_df['template psf good']) 
                                    & (median_df['calexp psf'] < median_df['template psf deep'])]

# Difference between the Calexp and Template PSF
diff_visit_good = df_interesting_visits['calexp psf'].values - df_interesting_visits['template psf good'].values
diff_visit_deep =  df_interesting_visits['template psf deep'].values - df_interesting_visits['calexp psf'].values

In [ ]:
bins=np.linspace(8, 19, 20)
plt.hist(df_interesting_visits['calexp psf'], bins=bins, alpha=1.0, edgecolor='k',  histtype='step', label='Visit')
plt.hist(df_interesting_visits['template psf deep'], bins=bins, alpha=1.0, edgecolor='orange',  histtype='step', label='Deep')
plt.hist(df_interesting_visits['template psf good'], bins=bins, alpha=1.0, edgecolor='blue',  histtype='step', label='Good')
plt.xlabel('PSF radius')
# plt.title('PSF historam of Template images')
plt.legend()

bins=np.linspace(0.0, 3.0, 20)
plt.hist(diff_visit_good, bins=bins, alpha=1.0, edgecolor='k',  histtype='step', label='Visit - Good template PSF')
plt.hist(diff_visit_deep, bins=bins, alpha=1.0, edgecolor='orange',  histtype='step', label='Deep template - Visit PSF')
plt.xlabel('PSF Difference radius')
# plt.title('PSF historam of Template images')
plt.legend()

bins=np.linspace(0.0, 0.5, 20)
plt.hist(diff_visit_good/df_interesting_visits['calexp psf'].values, bins=bins, alpha=1.0, edgecolor='k',  histtype='step', label='Visit - Good template PSF')
plt.hist(diff_visit_deep/df_interesting_visits['calexp psf'].values, bins=bins, alpha=1.0, edgecolor='orange',  histtype='step', label='Deep template - Visit PSF')
plt.xlabel('PSF Difference radius / Visit PSF')
# plt.title('PSF historam of Template images')
plt.legend()

![alt text](./PREOPS-4952-figures/psf_good_deep_calexp.png "Title")

![alt text](./PREOPS-4952-figures/psf_difference.png "Title")

![alt text](./PREOPS-4952-figures/psf_difference_fraction.png "Title")

### SNR of interesting visits